In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("./best.pt")

def detect_image(image):
    model = YOLO("./best.pt")  # 모델 로드
    results = model.predict(image, imgsz=1280, save=False)  # YOLO 예측 실행
    
    result_image = results[0].plot()  # 예측 결과 이미지 변환
    return result_image  # 결과 반환

# Gradio UI 설정
demo = gr.Interface(
    fn=detect_image,  # 이미지 분석 함수
    inputs=gr.Image(type="numpy"),  # 입력: 이미지 업로드
    outputs=gr.Image(type="numpy"),  # 출력: 분석된 이미지
    title="YOLO Image Detection",  # UI 제목
    description="Upload an image to detect objects using YOLO model."
)

demo.launch(share=True)  # UI 실행

Running on local URL:  http://127.0.0.1:7861

0: 960x1280 (no detections), 95.2ms
Speed: 7.9ms preprocess, 95.2ms inference, 3.7ms postprocess per image at shape (1, 3, 960, 1280)
WARNING ⚠️ 'source' is missing. Using 'source=/Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets'.

image 1/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/bus.jpg: 1280x960 1 pothole, 99.6ms
image 2/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/zidane.jpg: 736x1280 (no detections), 69.7ms
Speed: 3.5ms preprocess, 84.7ms inference, 2.3ms postprocess per image at shape (1, 3, 736, 1280)

Could not create share link. Missing file: /Users/k23070952/Library/Python/3.9/lib/python/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: ht

WARNING ⚠️ 'source' is missing. Using 'source=/Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets'.

image 1/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/bus.jpg: 1280x960 1 pothole, 94.7ms
image 2/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/zidane.jpg: 736x1280 (no detections), 60.0ms
Speed: 4.0ms preprocess, 77.4ms inference, 0.4ms postprocess per image at shape (1, 3, 736, 1280)
WARNING ⚠️ 'source' is missing. Using 'source=/Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets'.

image 1/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/bus.jpg: 1280x960 1 pothole, 91.5ms
image 2/2 /Users/k23070952/Library/Python/3.9/lib/python/site-packages/ultralytics/assets/zidane.jpg: 736x1280 (no detections), 58.8ms
Speed: 3.5ms preprocess, 75.2ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)


In [6]:
ls

README.md        best.pt          test.ipynb       yolov8n.pt
Test_images/     pothole_v8.yaml  train.ipynb
UI.ipynb         runs/            train.py


In [1]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
    
demo.launch(share=True, debug=True)

C:\Users\user\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Keyboard interruption in main thread... closing server.
